In [8]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

# 加载当前目录下的.env文件
# load_dotenv()
# load_dotenv(override=True) 会重新读取.env
load_dotenv(override=True)

# 现在可以像访问普通环境变量一样访问.env文件中的变量了
api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_API_BASE')

print(api_key)
print(base_url)

chat = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name="moonshot-v1-8k",
    temperature=0.7,
)

sk-UGVpjuTwo2Q8pewoqUDfckw1A0pbSDli9ElFMeS9WareKknG
https://api.moonshot.cn/v1/


In [1]:
from langchain_core.tools import tool


# 创建一个自定义工具

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

In [2]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(first_int: int, second_int: int) -> int - Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [3]:
multiply.invoke({"first_int": 4, "second_int": 5})

20

In [4]:
# 创建我们的提示
from langchain.tools.render import render_text_description

rendered_tools = render_text_description([multiply])
rendered_tools

'multiply: multiply(first_int: int, second_int: int) -> int - Multiply two integers together.'

In [10]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

print(prompt)

input_variables=['input'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nmultiply: multiply(first_int: int, second_int: int) -> int - Multiply two integers together.\n\nGiven the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]


In [9]:
# 添加输出解析器
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI

chain = prompt | chat | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})

{'name': 'multiply', 'arguments': {'first_int': 13, 'second_int': 4}}

In [11]:
# 从多种工具中选择

@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base ** exponent

In [ ]:
# 如果我们想运行模型选择工具，我们可以使用一个函数来返回基于模型输出的工具。具体来说，我们的函数将返回它自己的子链，
# 该子链获取模型输出的"参数"部分，并将其传递给所选的工具：
tools = [add, exponentiate, multiply]


def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

In [ ]:
rendered_tools = render_text_description(tools)
system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

chain = prompt | chat | JsonOutputParser() | tool_chain
chain.invoke({"input": "what's 3 plus 1132"})